In [19]:
import torch
import torch.nn as nn
import torch.onnx
import json
import os 
import ezkl

# Define the models
class MLP_Sigmoid(nn.Module):
    def __init__(self):
        super(MLP_Sigmoid, self).__init__()
        self.fc = nn.Linear(3, 3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

class MLP_Relu(nn.Module):
    def __init__(self):
        super(MLP_Relu, self).__init__()
        self.fc = nn.Linear(3, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        return x

class Conv_Sigmoid(nn.Module):
    def __init__(self):
        super(Conv_Sigmoid, self).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=3, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv(x)
        x = self.sigmoid(x)
        return x

class Conv_Relu(nn.Module):
    def __init__(self):
        super(Conv_Relu, self).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=3, stride=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

# Instantiate the models
mlp_sigmoid = MLP_Sigmoid()
mlp_relu = MLP_Relu()
conv_sigmoid = Conv_Sigmoid()
conv_relu = Conv_Relu()

# Dummy input tensor for mlp
dummy_input_mlp = torch.randn(1, 3)
input_mlp_path = 'mlp_input.json'

# Dummy input tensor for conv
dummy_input_conv = torch.randn(1, 1, 3)
input_conv_path = 'conv_input.json'

In [20]:
names = ['mlp_sigmoid', 'mlp_relu', 'conv_sigmoid', 'conv_relu']
models = [mlp_sigmoid, mlp_relu, conv_sigmoid, conv_relu]
inputs = [dummy_input_mlp, dummy_input_mlp, dummy_input_conv, dummy_input_conv]
input_paths = [input_mlp_path, input_mlp_path, input_conv_path, input_conv_path]

In [35]:
import subprocess


for name, model, x, input_path in zip(names, models, inputs, input_paths):
    # Create a new directory for the model if it doesn't exist
    if not os.path.exists(name):
        os.mkdir(name)
    # Store the paths in each of their respective directories
    model_path = os.path.join(name, "network.onnx")
    compiled_model_path = os.path.join(name, "network.compiled")
    pk_path = os.path.join(name, "test.pk")
    vk_path = os.path.join(name, "test.vk")
    settings_path = os.path.join(name, "settings.json")

    witness_path = os.path.join(name, "witness.json")
    # data_path = os.path.join(name, "input.json")
    sol_code_path = os.path.join(name, 'test.sol')
    sol_key_code_path = os.path.join(name, 'test_key.sol')
    abi_path = os.path.join(name, 'test.abi')

    # Flips the neural net into inference mode
    model.eval()

    # Export the model
    torch.onnx.export(model,               # model being run
                        x,                   # model input (or a tuple for multiple inputs)
                        model_path,            # where to save the model (can be a file or file-like object)
                        export_params=True,        # store the trained parameter weights inside the model file
                        opset_version=10,          # the ONNX version to export the model to
                        do_constant_folding=True,  # whether to execute constant folding for optimization
                        input_names = ['input'],   # the model's input names
                        output_names = ['output'], # the model's output names
                        dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                        'output' : {0 : 'batch_size'}})

    data_array = ((x).detach().numpy()).reshape([-1]).tolist()

    data = dict(input_data = [data_array])

        # Serialize data into file:
    json.dump(data, open(input_path, 'w' ))

    py_run_args = ezkl.PyRunArgs()
    py_run_args.input_visibility = "private"
    py_run_args.output_visibility = "public"
    py_run_args.param_visibility = "fixed" # private by default

    res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

    assert res == True

    await ezkl.calibrate_settings(input_path, model_path, settings_path, "resources")

    res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
    assert res == True

    res = await ezkl.get_srs( settings_path)
    assert res == True

    # now generate the witness file 

    res = await ezkl.gen_witness(input_path, compiled_model_path, witness_path)
    assert os.path.isfile(witness_path) == True

    res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

    assert res == True
    assert os.path.isfile(vk_path)
    assert os.path.isfile(pk_path)
    assert os.path.isfile(settings_path)

    res = await ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path,
        render_vk_seperately = True
    )
    assert res == True

    # Now set the vk path relative to execution from root directory
    # Define full path for the model directory
    absolute_root_path = os.path.abspath(".")
    # Run the following command from the command line within the notebook
    cmd = f"ezkl create-evm-vk --vk-path {os.path.abspath(vk_path)} --settings-path {os.path.abspath(settings_path)} --sol-code-path {os.path.abspath(sol_key_code_path)}"
    try:
        result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e.stderr}")
        print(f"Return code: {e.returncode}")
        print(f"Output: {e.stdout}")


Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 3 columns 

[*] [2024-06-17 21:45:53, ezkl] - 
 |  
 |          ███████╗███████╗██╗  ██╗██╗
 |          ██╔════╝╚══███╔╝██║ ██╔╝██║
 |          █████╗    ███╔╝ █████╔╝ ██║
 |          ██╔══╝   ███╔╝  ██╔═██╗ ██║
 |          ███████╗███████╗██║  ██╗███████╗
 |          ╚══════╝╚══════╝╚═╝  ╚═╝╚══════╝
 |  
 |          -----------------------------------------------------------
 |          Easy Zero Knowledge Learning.
 |          -----------------------------------------------------------
 |  
 |          
[*] [2024-06-17 21:45:53, ezkl] - command: 
 |   {
 |    "CreateEvmVK": {
 |      "abi_path": "vk.abi",
 |      "settings_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/mlp_sigmoid/settings.json",
 |      "sol_code_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/mlp_sigmoid/test_key.sol",
 |      "srs_path": null,
 |      "vk_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/mlp_sigmoid/test.vk"
 |    }
 |  }
[*] [2024-06-17 2



 <------------- Numerical Fidelity Report (input_scale: 11, param_scale: 12, scale_input_multiplier: 1) ------------->

+----------------+----------------+----------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error   | max_error      | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+----------------+----------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00012153387 | -0.00012153387 | -0.00012153387 | -0.00012153387 | 0.00012153387  | 0.00012153387    | 0.00012153387 | 0.00012153387 | 0.000000014770482  | -0.00029051772     | 0.00029051772          |
+----------------+----------------+----------------+------

[*] [2024-06-17 21:45:54, ezkl] - 
 |  
 |          ███████╗███████╗██╗  ██╗██╗
 |          ██╔════╝╚══███╔╝██║ ██╔╝██║
 |          █████╗    ███╔╝ █████╔╝ ██║
 |          ██╔══╝   ███╔╝  ██╔═██╗ ██║
 |          ███████╗███████╗██║  ██╗███████╗
 |          ╚══════╝╚══════╝╚═╝  ╚═╝╚══════╝
 |  
 |          -----------------------------------------------------------
 |          Easy Zero Knowledge for Layers.
 |          -----------------------------------------------------------
 |  
 |          
[*] [2024-06-17 21:45:54, ezkl] - command: 
 |   {
 |    "CreateEvmVK": {
 |      "abi_path": "vk.abi",
 |      "settings_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/mlp_relu/settings.json",
 |      "sol_code_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/mlp_relu/test_key.sol",
 |      "srs_path": null,
 |      "vk_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/mlp_relu/test.vk"
 |    }
 |  }
[*] [2024-06-17 21:45:54



 <------------- Numerical Fidelity Report (input_scale: 11, param_scale: 12, scale_input_multiplier: 1) ------------->

+----------------+----------------+----------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error   | max_error      | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+----------------+----------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00028786063 | -0.00028786063 | -0.00028786063 | -0.00028786063 | 0.00028786063  | 0.00028786063    | 0.00028786063 | 0.00028786063 | 0.00000008286374   | -0.00096739165     | 0.00096739165          |
+----------------+----------------+----------------+------

[*] [2024-06-17 21:45:54, ezkl] - 
 |  
 |          ███████╗███████╗██╗  ██╗██╗
 |          ██╔════╝╚══███╔╝██║ ██╔╝██║
 |          █████╗    ███╔╝ █████╔╝ ██║
 |          ██╔══╝   ███╔╝  ██╔═██╗ ██║
 |          ███████╗███████╗██║  ██╗███████╗
 |          ╚══════╝╚══════╝╚═╝  ╚═╝╚══════╝
 |  
 |          -----------------------------------------------------------
 |          Easy Zero Knowledge for Layers.
 |          -----------------------------------------------------------
 |  
 |          
[*] [2024-06-17 21:45:54, ezkl] - command: 
 |   {
 |    "CreateEvmVK": {
 |      "abi_path": "vk.abi",
 |      "settings_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/conv_sigmoid/settings.json",
 |      "sol_code_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/conv_sigmoid/test_key.sol",
 |      "srs_path": null,
 |      "vk_path": "/Users/ethancemer/Documents/development/ezkl/examples/notebooks/conv_sigmoid/test.vk"
 |    }
 |  }
[*] [2024-06